# タグ情報を利用した類似画像検索についての検討

#### Import modules

In [1]:
import os
import sys
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm
import mysql.connector
import tensorflow as tf
import tensorflow_hub as hub
from annoy import AnnoyIndex
import matplotlib.pyplot as plt

In [2]:
host = "localhost"
port = 3306
user = "root"
password = "humaninterface"
conn = mysql.connector.connect(
    host = host,
    port = port,
    user = user,
    password = password
)
image_dir = "/Volumes/Samsung_T5/19MH228/similar-image-retrieval/images/images_season_id_22"
image_ext = '.jpg'
image_path_list = glob.glob(image_dir + '/*' + image_ext)
image_ids = np.array([int(os.path.splitext(os.path.basename(image_path))[0]) for image_path in image_path_list])

#### Check the number of collection images

In [31]:
collection_news_id_statement = 'SELECT id FROM fp_production.news WHERE category_sub="collection";'
cur.execute(collection_news_id_statement)
collection_news_ids = [row[0] for row in cur.fetchall()]
collection_image_id_statement = 'SELECT id FROM fp_production.images WHERE new_id IN ({});'.format(','.join([str(news_id) for news_id in collection_news_ids]))
cur.execute(collection_image_id_statement)
collection_image_ids = [row[0] for row in cur.fetchall()]
print('the number of collection images -> {}'.format(len(collection_image_ids)))

the number of collection images -> 474199


#### Check the number of labeled collection images

In [32]:
labeled_image_id_statement = 'SELECT DISTINCT image_id FROM fp_production.image_details;'
cur.execute(labeled_image_id_statement)
labeled_image_ids = [row[0] for row in cur.fetchall()]
print('the number of labeled images -> {}'.format((len(labeled_image_ids))))

the number of labeled images -> 48479


<br>

In [4]:
cur = conn.cursor(buffered=True)

distinct_color_statement = 'SELECT DISTINCT color FROM fp_production.image_details;'
cur.execute(distinct_color_statement)
distinct_colors = [row[0] for row in cur.fetchall()]
distinct_colors = distinct_colors if '' in distinct_colors else distinct_colors + ['']
color_dict = {}
for i, color in enumerate(distinct_colors):
    color_dict[color] = i 

distinct_materials_statement = 'SELECT DISTINCT material FROM fp_production.image_details;'
cur.execute(distinct_materials_statement)
distinct_materials = [row[0] for row in cur.fetchall()]
distinct_materials = distinct_materials if '' in distinct_materials else distinct_materials + ['']
material_dict = {}
for i, material in enumerate(distinct_materials):
    material_dict[material] = i 

distinct_patterns_statement = 'SELECT DISTINCT pattern FROM fp_production.image_details;'
cur.execute(distinct_patterns_statement)
distinct_patterns = [row[0] for row in cur.fetchall()]
distinct_patterns = distinct_patterns if '' in distinct_patterns else distinct_patterns + ['']
pattern_dict = {}
for i, pattern in enumerate(distinct_patterns):
    pattern_dict[pattern] = i 

distinct_item_categories_statement = 'SELECT DISTINCT item_category FROM fp_production.image_details;'
cur.execute(distinct_item_categories_statement)
distinct_item_categories = [row[0] for row in cur.fetchall()]
distinct_item_categories = distinct_item_categories if '' in distinct_item_categories else distinct_item_categories + ['']
item_category_dict = {}
for i, item_category in enumerate(distinct_item_categories):
    item_category_dict[item_category] = i 

distinct_item_groups_statement = 'SELECT DISTINCT item_group FROM fp_production.image_details;'
cur.execute(distinct_item_groups_statement)
distinct_item_groups = [row[0] for row in cur.fetchall()]
distinct_item_groups = distinct_item_groups if '' in distinct_item_groups else distinct_item_groups + ['']
item_group_dict = {}
for i, item_group in enumerate(distinct_item_groups):
    item_group_dict[item_group] = i 

distinct_items_statement = 'SELECT DISTINCT item FROM fp_production.image_details;'
cur.execute(distinct_items_statement)
distinct_items = [row[0] for row in cur.fetchall()]
distinct_items = distinct_items if '' in distinct_items else distinct_items + ['']
item_dict = {}
for i, item in enumerate(distinct_items):
    item_dict[item] = i 
    
item_information_dict = {
    'color': color_dict,
    'material': material_dict,
    'pattern': pattern_dict,
    'item_category': item_category_dict,
    'item_group': item_group_dict,
    'item': item_dict
}

In [5]:
count = 0
for image_id in image_ids:
    print('image_id -> {}'.format(image_id))
    statement = "SELECT color, material, pattern, item_category, item_group, item, image_id FROM fp_production.image_details WHERE image_id = '{}';".format(image_id)
    cur = conn.cursor(buffered=True)
    cur.execute(statement)
    rows = cur.fetchall()
    for row in rows:
        color, material, pattern, item_category, item_group, item, image_id = row
        color_id = item_information_dict['color'][color]
        material_id = item_information_dict['material'][material]
        pattern_id = item_information_dict['pattern'][pattern]
        item_category_id = item_information_dict['item_category'][item_category]
        item_group_id = item_information_dict['item_group'][item_group]
        item_id = item_information_dict['item'][item]
        item_vector = [color_id, material_id, pattern_id, item_category_id, item_group_id, item_id]
        print('\titem_vector -> {}'.format(item_vector))
        count += 1

image_id -> 946023
	item_vector -> [2, 14, 2, 4, 20, 0]
	item_vector -> [2, 16, 2, 1, 19, 0]
image_id -> 956230
image_id -> 864296
image_id -> 880662
image_id -> 944634
image_id -> 972386
image_id -> 882075
	item_vector -> [0, 18, 2, 0, 1, 24]
	item_vector -> [4, 16, 18, 4, 7, 26]
	item_vector -> [0, 16, 2, 1, 19, 0]
	item_vector -> [0, 0, 0, 7, 11, 0]
image_id -> 882713
image_id -> 914765
image_id -> 952730
image_id -> 944152
image_id -> 948768
image_id -> 880104
	item_vector -> [14, 16, 16, 4, 20, 0]
image_id -> 916172
image_id -> 950127
	item_vector -> [0, 16, 16, 2, 3, 0]
	item_vector -> [2, 16, 2, 4, 7, 4]
image_id -> 929382
image_id -> 890317
	item_vector -> [9, 16, 16, 4, 24, 19]
	item_vector -> [6, 16, 2, 0, 1, 5]
	item_vector -> [2, 16, 2, 1, 19, 0]
image_id -> 891009
image_id -> 899455
image_id -> 966495
image_id -> 950133
	item_vector -> [0, 16, 2, 0, 1, 5]
	item_vector -> [2, 16, 16, 4, 7, 9]
	item_vector -> [0, 16, 2, 1, 2, 0]
image_id -> 916166
image_id -> 890303
	item_ve

image_id -> 933053
image_id -> 924249
	item_vector -> [2, 17, 16, 2, 3, 0]
image_id -> 973311
image_id -> 877605
	item_vector -> [8, 0, 0, 7, 11, 0]
	item_vector -> [2, 16, 18, 4, 7, 4]
	item_vector -> [2, 16, 16, 1, 19, 0]
image_id -> 935344
	item_vector -> [0, 17, 2, 0, 1, 15]
	item_vector -> [0, 16, 2, 1, 19, 0]
image_id -> 915094
image_id -> 922638
	item_vector -> [9, 16, 2, 2, 3, 0]
	item_vector -> [6, 16, 2, 4, 20, 0]
	item_vector -> [2, 16, 2, 4, 24, 13]
image_id -> 925157
image_id -> 921331
	item_vector -> [4, 16, 2, 2, 3, 0]
image_id -> 927998
image_id -> 896389
image_id -> 969349
image_id -> 928473
	item_vector -> [14, 14, 16, 2, 3, 0]
	item_vector -> [19, 0, 0, 5, 30, 0]
image_id -> 927740
	item_vector -> [8, 0, 0, 7, 11, 0]
	item_vector -> [5, 16, 2, 2, 3, 0]
image_id -> 873477
image_id -> 881839
image_id -> 921325
image_id -> 917697
image_id -> 937547
image_id -> 928467
	item_vector -> [10, 16, 16, 2, 3, 0]
	item_vector -> [19, 0, 0, 5, 30, 0]
image_id -> 872769
	item_vect

	item_vector -> [8, 16, 2, 0, 0, 29]
	item_vector -> [8, 16, 2, 1, 19, 0]
image_id -> 931692
image_id -> 947704
image_id -> 950278
image_id -> 948437
image_id -> 951172
image_id -> 855466
	item_vector -> [5, 16, 16, 2, 3, 0]
image_id -> 881151
	item_vector -> [2, 16, 2, 4, 7, 4]
	item_vector -> [12, 16, 18, 0, 0, 31]
	item_vector -> [2, 16, 2, 1, 19, 0]
image_id -> 930598
image_id -> 947710
image_id -> 948423
image_id -> 921495
image_id -> 878292
	item_vector -> [0, 0, 0, 5, 31, 0]
	item_vector -> [2, 16, 2, 2, 5, 0]
	item_vector -> [0, 16, 18, 0, 1, 30]
image_id -> 933091
image_id -> 913341
image_id -> 882458
image_id -> 870816
	item_vector -> [12, 16, 16, 0, 1, 15]
	item_vector -> [3, 16, 16, 4, 20, 0]
	item_vector -> [8, 16, 18, 4, 7, 4]
	item_vector -> [12, 16, 16, 1, 2, 0]
image_id -> 945107
image_id -> 895642
	item_vector -> [0, 16, 18, 4, 7, 4]
	item_vector -> [0, 17, 2, 0, 1, 5]
	item_vector -> [9, 17, 2, 1, 19, 0]
image_id -> 885451
	item_vector -> [2, 17, 16, 4, 20, 0]
	item_

image_id -> 932762
image_id -> 926717
	item_vector -> [9, 16, 16, 2, 3, 0]
	item_vector -> [0, 16, 16, 0, 1, 30]
image_id -> 960742
image_id -> 966333
image_id -> 930175
image_id -> 894827
image_id -> 933454
image_id -> 932992
	item_vector -> [0, 0, 0, 7, 11, 0]
	item_vector -> [2, 16, 2, 4, 7, 28]
	item_vector -> [20, 6, 16, 0, 1, 5]
	item_vector -> [9, 9, 2, 1, 19, 0]
image_id -> 902689
image_id -> 877564
	item_vector -> [5, 16, 2, 4, 20, 0]
	item_vector -> [9, 16, 2, 0, 1, 11]
	item_vector -> [9, 16, 16, 0, 1, 30]
	item_vector -> [9, 16, 16, 1, 19, 0]
image_id -> 922559
image_id -> 873702
	item_vector -> [2, 14, 2, 2, 3, 0]
image_id -> 968336
image_id -> 881594
image_id -> 880852
image_id -> 928712
image_id -> 928074
image_id -> 927347
image_id -> 920428
image_id -> 948780
image_id -> 931525
	item_vector -> [0, 16, 2, 2, 3, 0]
image_id -> 864518
image_id -> 925550
image_id -> 867011
image_id -> 882923
image_id -> 973716
image_id -> 877202
image_id -> 935743
	item_vector -> [14, 17, 

image_id -> 929550
image_id -> 884598
image_id -> 944970
image_id -> 933520
image_id -> 928666
image_id -> 927555
image_id -> 938475
image_id -> 948192
	item_vector -> [8, 16, 18, 0, 0, 12]
	item_vector -> [16, 16, 2, 4, 7, 4]
	item_vector -> [8, 16, 16, 1, 19, 0]
image_id -> 863465
image_id -> 941808
image_id -> 936458
	item_vector -> [9, 16, 16, 2, 3, 0]
image_id -> 931337
image_id -> 929578
	item_vector -> [12, 14, 16, 4, 7, 21]
	item_vector -> [4, 14, 16, 1, 19, 0]
image_id -> 887091
image_id -> 931451
image_id -> 959921
image_id -> 880098
image_id -> 967617
image_id -> 937020
image_id -> 928100
image_id -> 933246
	item_vector -> [0, 16, 2, 2, 3, 0]
image_id -> 871707
image_id -> 876068
	item_vector -> [0, 17, 18, 4, 7, 28]
	item_vector -> [0, 16, 2, 1, 19, 0]
image_id -> 943979
image_id -> 973662
image_id -> 882857
	item_vector -> [0, 6, 2, 0, 0, 6]
	item_vector -> [0, 14, 16, 4, 20, 0]
	item_vector -> [16, 16, 16, 1, 19, 0]
image_id -> 914821
image_id -> 870419
image_id -> 871713

...

image_id -> 929548
image_id -> 931307
image_id -> 863455
image_id -> 941838
image_id -> 874129
image_id -> 947291
image_id -> 936468
	item_vector -> [9, 16, 2, 4, 24, 13]
	item_vector -> [0, 16, 2, 1, 19, 0]
image_id -> 959911
image_id -> 931461
image_id -> 946831
image_id -> 928130
image_id -> 876058
	item_vector -> [8, 16, 2, 0, 1, 5]
	item_vector -> [8, 18, 2, 0, 1, 11]
	item_vector -> [0, 16, 2, 4, 7, 4]
	item_vector -> [0, 16, 2, 1, 19, 0]
image_id -> 943949
image_id -> 933276
	item_vector -> [0, 16, 2, 0, 1, 30]
	item_vector -> [0, 16, 18, 1, 19, 0]
image_id -> 932168
image_id -> 870429
image_id -> 882867
	item_vector -> [12, 16, 16, 0, 0, 12]
	item_vector -> [12, 6, 2, 4, 7, 28]
	item_vector -> [0, 16, 16, 1, 19, 0]
image_id -> 914811
image_id -> 962987
image_id -> 973652
image_id -> 877346
image_id -> 959939
image_id -> 880080
	item_vector -> [6, 16, 16, 4, 7, 28]
	item_vector -> [0, 16, 18, 1, 19, 0]
image_id -> 864474
image_id -> 946819
image_id -> 931449
image_id -> 928118
i

image_id -> 953178
image_id -> 882245
image_id -> 922959
image_id -> 882523
	item_vector -> [14, 17, 18, 0, 1, 24]
	item_vector -> [14, 17, 2, 0, 0, 29]
	item_vector -> [14, 17, 18, 1, 19, 0]
image_id -> 934685
	item_vector -> [16, 18, 16, 0, 1, 24]
	item_vector -> [14, 16, 2, 4, 7, 4]
	item_vector -> [14, 16, 2, 1, 2, 0]
image_id -> 944362
image_id -> 875815
image_id -> 890127
	item_vector -> [0, 16, 2, 4, 20, 0]
	item_vector -> [0, 0, 0, 5, 18, 0]
	item_vector -> [0, 16, 16, 0, 1, 24]
	item_vector -> [0, 16, 2, 1, 19, 0]
image_id -> 940104
image_id -> 950317
image_id -> 931925
image_id -> 968850
image_id -> 945054
image_id -> 972820
image_id -> 920800
	item_vector -> [0, 6, 2, 4, 7, 28]
	item_vector -> [2, 14, 16, 2, 3, 0]
image_id -> 899895
image_id -> 967593
image_id -> 947643
image_id -> 855535
	item_vector -> [0, 16, 16, 0, 0, 10]
	item_vector -> [2, 16, 18, 4, 7, 28]
image_id -> 928284
	item_vector -> [10, 16, 16, 4, 7, 26]
	item_vector -> [19, 16, 16, 1, 2, 0]
image_id -> 96887

image_id -> 945122
image_id -> 877184
image_id -> 953740
image_id -> 913370
image_id -> 945136
image_id -> 882469
	item_vector -> [0, 17, 2, 0, 1, 24]
	item_vector -> [0, 17, 2, 1, 19, 0]
image_id -> 877190
image_id -> 957532
image_id -> 948412
image_id -> 947721
image_id -> 855457
	item_vector -> [5, 16, 16, 4, 20, 0]
	item_vector -> [5, 16, 16, 1, 2, 0]
image_id -> 881160
	item_vector -> [2, 16, 2, 4, 7, 4]
	item_vector -> [6, 9, 2, 0, 0, 8]
	item_vector -> [6, 16, 2, 1, 19, 0]
image_id -> 875787
image_id -> 891373
image_id -> 941436
image_id -> 959279
image_id -> 881606
image_id -> 880518
image_id -> 947047
	item_vector -> [10, 16, 2, 2, 3, 0]
image_id -> 940728
	item_vector -> [4, 16, 16, 2, 3, 0]
image_id -> 873490
image_id -> 915067
image_id -> 883011
image_id -> 885460
	item_vector -> [8, 16, 16, 4, 7, 28]
	item_vector -> [3, 16, 2, 1, 19, 0]
image_id -> 964846
image_id -> 889272
image_id -> 927943
image_id -> 880530
image_id -> 940700
image_id -> 890723
image_id -> 933936
	item

image_id -> 882124
image_id -> 867816
image_id -> 928683
	item_vector -> [5, 16, 16, 2, 3, 0]
	item_vector -> [0, 16, 2, 0, 1, 11]
image_id -> 938490
image_id -> 960823
image_id -> 948177
	item_vector -> [2, 17, 18, 4, 24, 13]
	item_vector -> [16, 16, 2, 1, 19, 0]
image_id -> 863480
image_id -> 887074
	item_vector -> [3, 16, 18, 4, 7, 4]
	item_vector -> [2, 16, 18, 4, 7, 4]
	item_vector -> [2, 17, 2, 1, 19, 0]
image_id -> 876933
image_id -> 943022
image_id -> 953231
image_id -> 945453
image_id -> 944995
image_id -> 952649
image_id -> 924907
	item_vector -> [2, 16, 16, 4, 7, 4]
	item_vector -> [14, 16, 16, 1, 19, 0]
image_id -> 973687
image_id -> 915502
image_id -> 943744
image_id -> 864489
image_id -> 887712
	item_vector -> [6, 17, 16, 4, 20, 0]
	item_vector -> [0, 16, 2, 1, 19, 0]
image_id -> 947522
image_id -> 951340
	item_vector -> [0, 16, 16, 4, 20, 0]
	item_vector -> [0, 16, 2, 1, 19, 0]
image_id -> 875584
image_id -> 874842
image_id -> 940259
image_id -> 887706
	item_vector -> [9

image_id -> 952701
image_id -> 914754
image_id -> 882722
	item_vector -> [0, 6, 2, 0, 1, 22]
	item_vector -> [9, 9, 2, 4, 20, 0]
	item_vector -> [0, 6, 2, 1, 19, 0]
	item_vector -> [0, 0, 0, 7, 11, 0]
image_id -> 924697
image_id -> 884347
image_id -> 954364
image_id -> 871866
	item_vector -> [0, 14, 2, 4, 7, 28]
	item_vector -> [0, 6, 2, 1, 19, 0]
	item_vector -> [0, 0, 0, 7, 11, 0]
image_id -> 944177
image_id -> 882736
image_id -> 914740
image_id -> 952715
image_id -> 945269
image_id -> 924683
image_id -> 948995
image_id -> 899470
image_id -> 949453
image_id -> 916157
image_id -> 959898
image_id -> 950102
	item_vector -> [14, 16, 16, 4, 7, 28]
	item_vector -> [2, 16, 2, 4, 7, 9]
	item_vector -> [0, 18, 2, 1, 2, 0]
image_id -> 926094
image_id -> 890332
	item_vector -> [2, 16, 18, 4, 20, 0]
	item_vector -> [12, 17, 16, 0, 1, 5]
	item_vector -> [8, 16, 2, 1, 19, 0]
image_id -> 937827
image_id -> 880647
image_id -> 898008
	item_vector -> [3, 16, 16, 0, 0, 12]
	item_vector -> [0, 16, 2, 1,